# CAPSTONE PROJECT WEEK 3

## 1. Installing required libraries

In [8]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge lxml --yes
print('Libraries installed!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

Libraries installed!


## 2. Importing the libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json # library to handle JSON files

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

# Matplotlib and associated plotting modules
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib.colors as colors

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler, normalize, scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

print('Libraries imported!')

Libraries imported!


## 3. Scraping and retrieving data from the Wikipedia page into a Pandas DataFrame.

In [2]:
# Reading the Data

df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
print('Before dropping the rows, Dataframe Shape is:', df.shape)
df = df[df['Borough'] != 'Not assigned'] #Dropping the boolean values as mentioned
df = df.reset_index(drop=True) # Reseting the index 

# Changing 'not assigned'values in Neighborhoods Column
for i in df['Neighbourhood'].index:  
    if df['Neighbourhood'].loc[i] == 'Not assigned':
        df['Neighbourhood'].loc[i] = df['Borough'].loc[i]
    

df.head(20)


Before dropping the rows, Dataframe Shape is: (288, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## 4. Reading Geo Coordinates from Given Table

In [3]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.head()


df = pd.merge(df, geo_df, how='left',
        left_on='Postcode', right_on='Postal Code')
df.drop(labels='Postal Code', axis=1, inplace=True)
df.columns = ['Postcode','Borough','Neighborhood','Latitude','Longitude']
df.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


## 5. Grouping Neighborhoods in the same Borough

In [4]:
# Grouping the dataframe for single postcode column as requested in Week3 Assignment

df_grouped = df.groupby(['Postcode','Borough','Latitude','Longitude'])['Neighborhood'].agg([('Unique_Counts', 'nunique'), 
                                  ('Neighborhood', lambda x: ', '.join(set(x)))])
df_grouped = df_grouped.drop(columns = 'Unique_Counts')
df_grouped.reset_index(inplace=True)
df_grouped.sort_values('Postcode')
df_grouped = df_grouped[['Postcode','Borough','Neighborhood','Latitude','Longitude']]
df_grouped.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


## 6. Checking the shape of the current DataFrame

In [5]:
print('Before dropping the rows, Dataframe Shape is:', df.shape)
print('After dropping the rows, Dataframe Shape is:', df_grouped.shape)

Before dropping the rows, Dataframe Shape is: (211, 5)
After dropping the rows, Dataframe Shape is: (103, 5)


## 7. Creating a map for city of Toronto with all the boroughs.

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


map_canada = folium.Map(
    location=[latitude,longitude],
    tiles='OpenStreetMap',
    zoom_start=11,
    )
df_grouped.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]]).add_to(map_canada), axis=1)
map_canada


The geograpical coordinate of Toronto are 43.653963, -79.387207.
